# Concierge Agent: Your Personal AI Travel Orchestrator

## 1. The Problem
**Travel planning is broken.**
Planning a single trip today requires juggling dozens of disconnected platforms: Expedia for flights, Airbnb for stays, TripAdvisor for activities, government sites for visas, and weather apps for packing.
*   **Information Overload**: Users are drowning in tabs and conflicting reviews.
*   **Context Switching**: Data doesn't flow between apps.
*   **Decision Fatigue**: The stress of coordinating logistics often overshadows the excitement of travel.

## 2. The Solution
**Concierge Agent: A Multi-Agent AI Orchestrator.**
We built a collaborative workforce of specialized AI agents powered by **Google's Agent Development Kit (ADK)** and **Gemini** models.
*   **Unified Experience**: Replace 10+ apps with one conversation.
*   **Active Orchestration**: Proactively checks requirements (visas, weather) based on your bookings.
*   **15+ Capabilities**: From planning and booking to real-time translation and emergency support.

## 3. Architecture
The system follows a hub-and-spoke **Multi-Agent Architecture** orchestrated by a Root Coordinator.

- **Root Coordinator**: The "Head Concierge" that understands intent and delegates tasks.
- **Planning Agent**: Crafts personalized itineraries.
- **Booking Agent**: Handles flights, hotels, and rides.
- **Utility Agent**: Manages logistics (visas, currency, insurance).
- **Search Agent**: Fetches real-time data (news, events) via Google Search.
- **Social Agent**: Manages user preferences and feedback.

## 4. How to Run This Notebook
1.  **Install Dependencies**: Run the first code cell to install `google-adk`.
2.  **Set API Key**: Ensure your `GOOGLE_API_KEY` is set in the environment or the `.env` file.
3.  **Run All Cells**: Execute all cells to define the agents and tools.
4.  **Launch**: The final cell will start the ADK web server.


In [1]:
# Install ADK if not already installed (Uncomment if needed)
# !pip install google-adk


In [2]:
import os
import json
import asyncio
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    print("Warning: GOOGLE_API_KEY not found in environment. Please set it.")
    # os.environ["GOOGLE_API_KEY"] = "YOUR_KEY_HERE"
else:
    print("✅ Google API Key loaded.")


✅ Google API Key loaded.


In [3]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")


✅ ADK components imported successfully.


## 1. Define Tools (Mock Implementations)
We define the 15 capabilities as standard Python functions.

In [4]:
# --- Planning Tools ---
def suggest_destinations(budget: str, season: str, interests: str):
    """Suggests travel destinations based on user preferences."""
    return json.dumps([
        {"city": "Kyoto", "country": "Japan", "reason": "Autumn foliage", "cost": "Medium"},
        {"city": "Reykjavik", "country": "Iceland", "reason": "Northern lights", "cost": "High"},
        {"city": "Bali", "country": "Indonesia", "reason": "Beaches", "cost": "Low"}
    ])

def create_itinerary(destination: str, days: int):
    """Creates a day-by-day itinerary."""
    return json.dumps({f"Day {i}": f"Activity in {destination}" for i in range(1, days + 1)})

def suggest_activities(city: str, interests: str):
    """Suggests activities in a city."""
    return json.dumps(["Museum Tour", "Hiking", "Food Tasting"])

# --- Booking Tools ---
def search_flights(origin: str, destination: str, date: str):
    """Searches for flights."""
    return json.dumps([{"airline": "AirFly", "price": 500, "id": "FL123"}])

def book_flight(flight_id: str, passenger_name: str):
    """Books a flight."""
    return json.dumps({"status": "confirmed", "ref": "BK-FL-001"})

def search_hotels(city: str, check_in: str):
    """Searches for hotels."""
    return json.dumps([{"name": "Grand Hotel", "price": 200, "id": "HTL1"}])

def book_hotel(hotel_id: str, guest_name: str):
    """Books a hotel."""
    return json.dumps({"status": "confirmed", "ref": "BK-HTL-001"})

def book_ride(pickup: str, dropoff: str):
    """Books a local ride."""
    return json.dumps({"driver": "John", "eta": "5 mins"})

def book_activity(activity_name: str, date: str):
    """Books an activity ticket."""
    return json.dumps({"status": "confirmed", "ticket": "ACT-001"})

# --- Utility Tools ---
def get_weather_forecast(city: str):
    """Gets weather forecast."""
    return "Sunny, 25°C"

def convert_currency(amount: float, from_curr: str, to_curr: str):
    """Converts currency."""
    return f"{amount * 1.1:.2f} {to_curr}"

def translate_text(text: str, target_lang: str):
    """Translates text."""
    return f"[Translated to {target_lang}]: {text}"

def check_visa_requirements(citizenship: str, country: str):
    """Checks visa requirements."""
    return "Visa-free for 90 days (Simulated)"

def get_insurance_quote(destination: str, days: int):
    """Gets travel insurance quote."""
    return "$50 Standard Plan"

def get_emergency_contacts(city: str):
    """Gets emergency contacts."""
    return "Police: 911, Embassy: +1-555-0199"

def get_flight_status(flight_number: str):
    """Checks flight status."""
    return "On Time"

def track_expense(item: str, amount: float):
    """Logs an expense."""
    return "Expense logged."

def get_budget_summary():
    """Returns total expenses."""
    return "Total: $150"

# --- Social Tools ---
user_prefs = {}
def update_user_preference(key: str, value: str):
    """Updates user preference."""
    user_prefs[key] = value
    return "Updated."

def get_user_preferences():
    """Gets user preferences."""
    return json.dumps(user_prefs)

def submit_feedback(rating: int, comment: str):
    """Submits feedback."""
    return "Feedback received."

def share_to_social_media(platform: str, content: str):
    """Shares content to social media."""
    return f"Shared to {platform}."


In [5]:
# --- Helper Utilities for Session and Metrics ---



# Simple Session Service (in-memory)

class SimpleSessionService:

    def __init__(self):

        self.sessions = {}

    def get(self, session_id):

        return self.sessions.get(session_id, {})

    def set(self, session_id, state):

        self.sessions[session_id] = state

    def clear(self, session_id):

        self.sessions.pop(session_id, None)



session_service = SimpleSessionService()



# Context Compaction (simple token limit)

def compact_context(messages, max_tokens=500):

    """Very naive compaction: keep last N messages"""

    if len(messages) <= max_tokens:

        return messages

    return messages[-max_tokens:]



# Metrics counters

class Metrics:

    def __init__(self):

        self.counters = {}

    def inc(self, name, amount=1):

        self.counters[name] = self.counters.get(name, 0) + amount

    def report(self):

        return self.counters



metrics = Metrics()



print("✅ Helper utilities initialized (session_service, metrics, compact_context)")

✅ Helper utilities initialized (session_service, metrics, compact_context)


## 2. Define Sub-Agents
We group the tools into specialized agents.

In [6]:
# Planning Agent
planning_agent = Agent(
    name="PlanningAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Travel Planner. Use your tools to suggest destinations, create itineraries, and suggest activities.
    IMPORTANT: After using a tool, you MUST provide a text summary of the results to the user. Do not just return the tool output.""",
    tools=[FunctionTool(suggest_destinations), FunctionTool(create_itinerary), FunctionTool(suggest_activities)],
    output_key="planning_output"
)

# Booking Agent
booking_agent = Agent(
    name="BookingAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Booking Specialist. Use your tools to search and book flights, hotels, rides, and activities.
    IMPORTANT: After using a tool, you MUST provide a text confirmation or summary of the booking details to the user.""",
    tools=[
        FunctionTool(search_flights), FunctionTool(book_flight),
        FunctionTool(search_hotels), FunctionTool(book_hotel),
        FunctionTool(book_ride), FunctionTool(book_activity)
    ],
    output_key="booking_output"
)

# Utility Agent (Mock Tools)
utility_agent = Agent(
    name="UtilityAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Travel Assistant. Provide info on weather, currency, visa, insurance, emergency contacts, and flight status using your specific tools.
    IMPORTANT: After using a tool, you MUST provide a clear text report of the information to the user.""",
    tools=[
        FunctionTool(get_weather_forecast), FunctionTool(convert_currency),
        FunctionTool(translate_text), FunctionTool(check_visa_requirements),
        FunctionTool(get_insurance_quote), FunctionTool(get_emergency_contacts),
        FunctionTool(get_flight_status), FunctionTool(track_expense),
        FunctionTool(get_budget_summary)
    ],
    output_key="utility_output"
)

# Search Agent (Real-time Tools)
# We separate this to avoid mixing FunctionTool and google_search which causes compatibility issues.
search_agent = Agent(
    name="SearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a Real-Time Information Specialist. Use `google_search` to find up-to-date information on:
    - Currency exchange rates
    - Real-time weather conditions
    - Visa requirements and travel advisories
    - Local events and news
    Always summarize the search results clearly for the user.""",
    tools=[google_search],
    output_key="search_output"
)

# Social Agent
social_agent = Agent(
    name="SocialAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You handle User Profile and Socials. Update preferences, collect feedback, and share updates.
    IMPORTANT: After using a tool, you MUST provide a text confirmation of the action to the user.""",
    tools=[
        FunctionTool(update_user_preference), FunctionTool(get_user_preferences),
        FunctionTool(submit_feedback), FunctionTool(share_to_social_media)
    ],
    output_key="social_output"
)

print("✅ Sub-agents created. SearchAgent separated for compatibility.")


✅ Sub-agents created. SearchAgent separated for compatibility.


## 3. Define Root Coordinator Agent
This agent orchestrates the others.

In [7]:
root_agent = Agent(
    name="ConciergeCoordinator",
    model="gemini-2.5-flash-lite",
    instruction="""You are the Head Concierge. Your goal is to assist the user with their travel needs by coordinating with specialized agents.
    - For planning (destinations, itineraries), call `PlanningAgent`.
    - For bookings (flights, hotels, rides), call `BookingAgent`.
    - For specific utilities (simulated weather, simulated currency, expenses), call `UtilityAgent`.
    - For REAL-TIME information (actual currency rates, actual weather, news), call `SearchAgent`.
    - For social/profile (preferences, feedback, sharing), call `SocialAgent`.
    
    Always answer the user politely. If a sub-agent returns information, summarize it for the user and ask if they need anything else.""",
    tools=[
        AgentTool(planning_agent),
        AgentTool(booking_agent),
        AgentTool(utility_agent),
        AgentTool(search_agent),
        AgentTool(social_agent)
    ]
)

print("✅ Root Coordinator created.")


✅ Root Coordinator created.


## 4. Run the Agent (Conversational Mode)
We use `InMemoryRunner` to execute a conversational loop.

In [8]:
# --- Missing Feature Implementations ---
# 1. Mock MCP Tool (Multi-Component Process)
class MCPTool(FunctionTool):
    def __init__(self, name: str):
        super().__init__(self.run)
        self.name = name
    def run(self, *args, **kwargs):
        # Simulate a multi-step process
        print(f'MCP {self.name} called with args={args}, kwargs={kwargs}')
        return f'MCP {self.name} completed'

# 2. OpenAPI Tool (mock)
def call_openapi(endpoint: str, payload: dict):
    print(f'OpenAPI Call to endpoint={endpoint}')
    # In a real scenario you would use requests.post...
    return {'status': 'success', 'data': payload}

# 3. Simple Session Service (in-memory)
class SimpleSessionService:
    def __init__(self):
        self.sessions = {}
    def get(self, session_id):
        return self.sessions.get(session_id, {})
    def set(self, session_id, state):
        self.sessions[session_id] = state
    def clear(self, session_id):
        self.sessions.pop(session_id, None)

session_service = SimpleSessionService()

# 4. Context Compaction (simple token limit)
def compact_context(messages, max_tokens=500):
    # Very naive compaction: keep last N messages
    if len(messages) <= max_tokens:
        return messages
    return messages[-max_tokens:]

# 5. Metrics (simple counters)
class Metrics:
    def __init__(self):
        self.counters = {}
    def inc(self, name, amount=1):
        self.counters[name] = self.counters.get(name, 0) + amount
    def report(self):
        return self.counters

metrics = Metrics()

# 6. Agent Evaluation (simple scoring)
def evaluate_response(response: str) -> float:
    # Placeholder: reward length and presence of keywords
    score = len(response) * 0.01
    for kw in ['success', 'confirmed', 'done']:
        if kw in response.lower():
            score += 0.5
    return min(score, 1.0)

# 7. Register new tools with agents
mcp_tool = MCPTool('example_mcp')
openapi_tool = FunctionTool(lambda endpoint, payload: call_openapi(endpoint, payload))



In [9]:
# --- Long‑running operation helpers (pause / resume) ---
import asyncio

# Simple flag‑based pause/resume for agents
class AgentPauseController:
    def __init__(self):
        self.paused = False
    async def pause(self):
        self.paused = True
        print('🛑 Agent execution paused')
    async def resume(self):
        self.paused = False
        print('▶️ Agent execution resumed')
    async def wait_if_paused(self):
        while self.paused:
            await asyncio.sleep(0.5)

# Create a global controller that can be used by any agent
agent_pause_controller = AgentPauseController()

# Expose as FunctionTool so agents can request pause/resume
pause_tool = FunctionTool(lambda: asyncio.run(agent_pause_controller.pause()))
resume_tool = FunctionTool(lambda: asyncio.run(agent_pause_controller.resume()))
# You can add these tools to any agent's tool list if needed
# Example: planning_agent.tools.append(pause_tool)
#          planning_agent.tools.append(resume_tool)
print('✅ Pause/Resume utilities added')


✅ Pause/Resume utilities added


In [10]:
# --- Long‑running operation helpers (pause / resume) ---
import asyncio

# Simple flag‑based pause/resume for agents
class AgentPauseController:
    def __init__(self):
        self.paused = False
    async def pause(self):
        self.paused = True
        print('🛑 Agent execution paused')
    async def resume(self):
        self.paused = False
        print('▶️ Agent execution resumed')
    async def wait_if_paused(self):
        while self.paused:
            await asyncio.sleep(0.5)

# Create a global controller that can be used by any agent
agent_pause_controller = AgentPauseController()

# Expose as FunctionTool so agents can request pause/resume
pause_tool = FunctionTool(lambda: asyncio.run(agent_pause_controller.pause()))
resume_tool = FunctionTool(lambda: asyncio.run(agent_pause_controller.resume()))
# You can add these tools to any agent's tool list if needed
# Example: planning_agent.tools.append(pause_tool)
#          planning_agent.tools.append(resume_tool)
print('✅ Pause/Resume utilities added')


✅ Pause/Resume utilities added


## MCP (Model Context Protocol) Implementation

This section implements a complete MCP server that exposes all 23 concierge tools.

**Benefits:**
- Standardized protocol for AI tools
- Better interoperability
- Production-ready deployment


In [11]:
# MCP Server - Define tool handlers
from mcp.server import Server
from mcp.types import Tool, TextContent
import mcp.server.stdio

# Create server instance
mcp_server = Server("concierge-agent")

print("✅ MCP Server instance created")


✅ MCP Server instance created


In [12]:
@mcp_server.call_tool()
async def call_tool(name: str, arguments: dict) -> list[TextContent]:
    """Handle tool calls from MCP clients"""
    
    # Planning Tools
    if name == "suggest_destinations":
        result = suggest_destinations(arguments.get("budget", ""), arguments.get("season", ""), arguments.get("interests", ""))
        return [TextContent(type="text", text=result)]
    elif name == "create_itinerary":
        result = create_itinerary(arguments.get("destination", ""), arguments.get("days", 1))
        return [TextContent(type="text", text=result)]
    elif name == "suggest_activities":
        result = suggest_activities(arguments.get("city", ""), arguments.get("interests", ""))
        return [TextContent(type="text", text=result)]
    
    # Booking Tools
    elif name == "search_flights":
        result = search_flights(arguments.get("origin", ""), arguments.get("destination", ""), arguments.get("date", ""))
        return [TextContent(type="text", text=result)]
    elif name == "book_flight":
        result = book_flight(arguments.get("flight_id", ""), arguments.get("passenger_name", ""))
        return [TextContent(type="text", text=result)]
    elif name == "search_hotels":
        result = search_hotels(arguments.get("city", ""), arguments.get("check_in", ""))
        return [TextContent(type="text", text=result)]
    elif name == "book_hotel":
        result = book_hotel(arguments.get("hotel_id", ""), arguments.get("guest_name", ""))
        return [TextContent(type="text", text=result)]
    elif name == "book_ride":
        result = book_ride(arguments.get("pickup", ""), arguments.get("dropoff", ""))
        return [TextContent(type="text", text=result)]
    elif name == "book_activity":
        result = book_activity(arguments.get("activity_name", ""), arguments.get("date", ""))
        return [TextContent(type="text", text=result)]
    
    # Utility Tools
    elif name == "get_weather_forecast":
        result = get_weather_forecast(arguments.get("city", ""))
        return [TextContent(type="text", text=result)]
    elif name == "convert_currency":
        result = convert_currency(arguments.get("amount", 0.0), arguments.get("from_curr", ""), arguments.get("to_curr", ""))
        return [TextContent(type="text", text=result)]
    elif name == "translate_text":
        result = translate_text(arguments.get("text", ""), arguments.get("target_lang", ""))
        return [TextContent(type="text", text=result)]
    elif name == "check_visa_requirements":
        result = check_visa_requirements(arguments.get("citizenship", ""), arguments.get("country", ""))
        return [TextContent(type="text", text=result)]
    elif name == "get_insurance_quote":
        result = get_insurance_quote(arguments.get("destination", ""), arguments.get("days", 1))
        return [TextContent(type="text", text=result)]
    elif name == "get_emergency_contacts":
        result = get_emergency_contacts(arguments.get("city", ""))
        return [TextContent(type="text", text=result)]
    elif name == "get_flight_status":
        result = get_flight_status(arguments.get("flight_number", ""))
        return [TextContent(type="text", text=result)]
    elif name == "track_expense":
        result = track_expense(arguments.get("item", ""), arguments.get("amount", 0.0))
        return [TextContent(type="text", text=result)]
    elif name == "get_budget_summary":
        result = get_budget_summary()
        return [TextContent(type="text", text=result)]
    
    # Social Tools
    elif name == "update_user_preference":
        result = update_user_preference(arguments.get("key", ""), arguments.get("value", ""))
        return [TextContent(type="text", text=result)]
    elif name == "get_user_preferences":
        result = get_user_preferences()
        return [TextContent(type="text", text=result)]
    elif name == "submit_feedback":
        result = submit_feedback(arguments.get("rating", 0), arguments.get("comment", ""))
        return [TextContent(type="text", text=result)]
    elif name == "share_to_social_media":
        result = share_to_social_media(arguments.get("platform", ""), arguments.get("content", ""))
        return [TextContent(type="text", text=result)]
    
    else:
        raise ValueError(f"Unknown tool: {name}")

print("✅ MCP tool handler registered (23 tools)")


✅ MCP tool handler registered (23 tools)


In [13]:
@mcp_server.list_tools()
async def list_tools() -> list[Tool]:
    """List all available tools with schemas"""
    return [
        # Planning Tools (3)
        Tool(name="suggest_destinations", description="Suggests travel destinations",
             inputSchema={"type": "object", "properties": {
                 "budget": {"type": "string"}, "season": {"type": "string"}, "interests": {"type": "string"}},
                 "required": ["budget", "season", "interests"]}),
        Tool(name="create_itinerary", description="Creates day-by-day itinerary",
             inputSchema={"type": "object", "properties": {
                 "destination": {"type": "string"}, "days": {"type": "integer"}},
                 "required": ["destination", "days"]}),
        Tool(name="suggest_activities", description="Suggests activities in a city",
             inputSchema={"type": "object", "properties": {
                 "city": {"type": "string"}, "interests": {"type": "string"}},
                 "required": ["city", "interests"]}),
        
        # Booking Tools (6)
        Tool(name="search_flights", description="Searches for flights",
             inputSchema={"type": "object", "properties": {
                 "origin": {"type": "string"}, "destination": {"type": "string"}, "date": {"type": "string"}},
                 "required": ["origin", "destination", "date"]}),
        Tool(name="book_flight", description="Books a flight",
             inputSchema={"type": "object", "properties": {
                 "flight_id": {"type": "string"}, "passenger_name": {"type": "string"}},
                 "required": ["flight_id", "passenger_name"]}),
        Tool(name="search_hotels", description="Searches for hotels",
             inputSchema={"type": "object", "properties": {
                 "city": {"type": "string"}, "check_in": {"type": "string"}},
                 "required": ["city", "check_in"]}),
        Tool(name="book_hotel", description="Books a hotel",
             inputSchema={"type": "object", "properties": {
                 "hotel_id": {"type": "string"}, "guest_name": {"type": "string"}},
                 "required": ["hotel_id", "guest_name"]}),
        Tool(name="book_ride", description="Books local transportation",
             inputSchema={"type": "object", "properties": {
                 "pickup": {"type": "string"}, "dropoff": {"type": "string"}},
                 "required": ["pickup", "dropoff"]}),
        Tool(name="book_activity", description="Books an activity",
             inputSchema={"type": "object", "properties": {
                 "activity_name": {"type": "string"}, "date": {"type": "string"}},
                 "required": ["activity_name", "date"]}),
        
        # Utility Tools (9)
        Tool(name="get_weather_forecast", description="Gets weather forecast",
             inputSchema={"type": "object", "properties": {"city": {"type": "string"}}, "required": ["city"]}),
        Tool(name="convert_currency", description="Converts currency",
             inputSchema={"type": "object", "properties": {
                 "amount": {"type": "number"}, "from_curr": {"type": "string"}, "to_curr": {"type": "string"}},
                 "required": ["amount", "from_curr", "to_curr"]}),
        Tool(name="translate_text", description="Translates text",
             inputSchema={"type": "object", "properties": {
                 "text": {"type": "string"}, "target_lang": {"type": "string"}},
                 "required": ["text", "target_lang"]}),
        Tool(name="check_visa_requirements", description="Checks visa requirements",
             inputSchema={"type": "object", "properties": {
                 "citizenship": {"type": "string"}, "country": {"type": "string"}},
                 "required": ["citizenship", "country"]}),
        Tool(name="get_insurance_quote", description="Gets insurance quote",
             inputSchema={"type": "object", "properties": {
                 "destination": {"type": "string"}, "days": {"type": "integer"}},
                 "required": ["destination", "days"]}),
        Tool(name="get_emergency_contacts", description="Gets emergency contacts",
             inputSchema={"type": "object", "properties": {"city": {"type": "string"}}, "required": ["city"]}),
        Tool(name="get_flight_status", description="Checks flight status",
             inputSchema={"type": "object", "properties": {"flight_number": {"type": "string"}}, "required": ["flight_number"]}),
        Tool(name="track_expense", description="Logs an expense",
             inputSchema={"type": "object", "properties": {
                 "item": {"type": "string"}, "amount": {"type": "number"}},
                 "required": ["item", "amount"]}),
        Tool(name="get_budget_summary", description="Gets budget summary",
             inputSchema={"type": "object", "properties": {}}),
        
        # Social Tools (4)
        Tool(name="update_user_preference", description="Updates user preference",
             inputSchema={"type": "object", "properties": {
                 "key": {"type": "string"}, "value": {"type": "string"}},
                 "required": ["key", "value"]}),
        Tool(name="get_user_preferences", description="Gets user preferences",
             inputSchema={"type": "object", "properties": {}}),
        Tool(name="submit_feedback", description="Submits feedback",
             inputSchema={"type": "object", "properties": {
                 "rating": {"type": "integer"}, "comment": {"type": "string"}},
                 "required": ["rating", "comment"]}),
        Tool(name="share_to_social_media", description="Shares to social media",
             inputSchema={"type": "object", "properties": {
                 "platform": {"type": "string"}, "content": {"type": "string"}},
                 "required": ["platform", "content"]}),
    ]

print("✅ MCP tool schemas registered (23 tools total)")


✅ MCP tool schemas registered (23 tools total)


### Using the MCP Server

The MCP server is now configured. To use it:

**Option 1: Export and run standalone**
```python
# Save MCP server code to mcp_server.py and run:
# python mcp_server.py
```

**Option 2: Use with ADK (shown below)**

The agents below use FunctionTool for simplicity. For production, consider using McpToolset.


## Running the Concierge Agent with ADK GUI



After executing all cells above, you can access the agent through the ADK web interface:



### Option 1: Using app.py (Recommended)

```bash

# In terminal, run:

python -m google.adk.cli web

```



Then open your browser to the URL shown (typically http://localhost:8000)



### Option 2: Direct from Notebook

If you want to create an app directly in this notebook, run the cell below.

## Interactive Chat in Notebook

Execute the cell below to chat with your Concierge Agent directly in this notebook.

**Note**: For web UI access, the `app.py` file in this directory is already configured. Just run:
```bash
python -m google.adk.cli web
```
Then open http://127.0.0.1:8000


In [ ]:
# Launch ADK Web Interface (Custom Script)
import subprocess
import sys
import time
import webbrowser
import threading
import os
from pathlib import Path

def run_adk_server():
    print("🚀 Preparing ADK Web Server...")
    
    # Create the custom runner script
    runner_script = Path("run_gui_custom.py")
    
    with open(runner_script, "w", encoding="utf-8") as f:
        f.write('''
import sys
import os
from pathlib import Path
import uvicorn

# Add current directory to path
sys.path.insert(0, str(Path(".").absolute()))

try:
    # Import correct function
    from google.adk.cli.fast_api import get_fast_api_app
    print("✅ Imported get_fast_api_app")
except ImportError as e:
    print(f"❌ Failed to import get_fast_api_app: {e}")
    sys.exit(1)

if __name__ == "__main__":
    print("🚀 Starting Fixed ADK Server...")
    
    try:
        # Create web app using the correct API
        # We point agents_dir to current directory so it finds app.py
        app = get_fast_api_app(
            agents_dir=".",
            web=True,
            host="127.0.0.1",
            port=8000
        )
        print("✅ App created successfully")
        
        print("   URL: http://127.0.0.1:8000")
        
        # Run with uvicorn
        uvicorn.run(app, host="127.0.0.1", port=8000)
        
    except Exception as e:
        print(f"❌ Error starting server: {e}")
        import traceback
        traceback.print_exc()
''')
    
    print(f"   Created runner: {runner_script}")
    print("   URL: http://127.0.0.1:8000")
    
    # Open browser automatically
    def open_browser():
        time.sleep(4)
        print("   Opening browser...")
        webbrowser.open("http://127.0.0.1:8000")
    
    threading.Thread(target=open_browser, daemon=True).start()
    
    # Run the custom script
    cmd = [sys.executable, "run_gui_custom.py"]
    
    try:
        # Check port
        import socket
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('127.0.0.1', 8000)) == 0:
                print("\n⚠️ Port 8000 seems to be in use!")
                print("   Please stop other running servers first.")
                return

        print(f"   Executing: {' '.join(cmd)}")
        subprocess.run(cmd, check=True)
        
    except KeyboardInterrupt:
        print("\n✅ Server stopped by user.")
    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        if runner_script.exists():
            try:
                runner_script.unlink()
                print(f"   Cleaned up {runner_script}")
            except:
                pass

if __name__ == "__main__":
    run_adk_server()


🚀 Preparing ADK Web Server...
   Created runner: run_gui_custom.py
   URL: http://127.0.0.1:8000

⚠️ Port 8000 seems to be in use!
   Please stop other running servers first.
   Cleaned up run_gui_custom.py


   Opening browser...


### Example Input :
i am planning a family vacation to Denmark with my wife and 2 kids under 5 years. for 7 days. i will be travelling from Pune. i have a estimated budget of 10,00000 INR. Share the itenary, weather condition,suggest best hotel to staly along with the weather condition. include Visa process as weel


### Output: 

It sounds like a wonderful family trip to Denmark! Planning a vacation for a family with young children requires careful consideration of activities, pacing, and accommodations. Here's a detailed itinerary, along with information on weather, hotels, and the visa process, tailored for your family of four from Pune with a budget of 10,00,000 INR for 7 days.

Family Vacation to Denmark with Young Children (7 Days)
Target Audience: Family of 4 (2 adults, 2 children under 5) Origin: Pune, India Duration: 7 Days Budget: 10,00,000 INR (approximately €11,000 - €12,000, depending on exchange rates)

1. Visa Process (Schengen Visa)
Denmark is part of the Schengen Area, so you will need a Schengen Visa.

Type of Visa: Short-stay Schengen Visa (Type C) for tourism.
Where to Apply: You'll need to apply through the Embassy of Denmark or their authorized visa application center (like VFS Global) in India. For Pune, you will likely need to visit the VFS Global center in Mumbai or a city with a Danish diplomatic mission.
When to Apply: Apply at least 6-8 weeks before your intended travel date.
Key Documents (General List, subject to change - always check the official website):
Visa Application Form: Duly filled and signed.
Passport: Valid for at least 3 months beyond your intended stay, with at least two blank pages.
Photographs: Recent passport-sized photographs meeting Schengen specifications.
Flight Itinerary: Round-trip flight bookings.
Proof of Accommodation: Hotel bookings for your entire stay.
Travel Medical Insurance: Valid for the entire Schengen area, covering at least €30,000 for medical emergencies and repatriation.
Proof of Financial Means:
Bank statements for the last 3-6 months (personal and/or joint accounts).
Payslips (if employed).
Income Tax Returns (ITRs) for the last 2-3 years.
A letter from your employer stating your position, salary, and duration of employment.
If self-employed, business registration documents, bank statements, and ITRs.
For Children:
Birth certificate.
Consent letter from both parents if traveling with only one parent.
Copies of parents' passports.
Cover Letter: Explaining the purpose of your visit, your itinerary, and how you will fund the trip.
Proof of Ties to India: Evidence that you will return to India (e.g., property deeds, family ties).
Visa Fee: Check the official website for the current fee (around €80 per adult, with possible concessions for children).
Biometrics: You will likely need to provide fingerprints and a photograph at the visa application center.
Important Note: Always refer to the official website of the Royal Danish Embassy in India or the relevant VFS Global website for the most up-to-date and precise requirements.
2. Best Time to Visit & Weather Conditions
For a family with young children, late spring (May-June) or early autumn (September) are generally ideal.

Summer (June-August):
Weather: Warmest months. Average temperatures: 15-25°C (59-77°F). Long daylight hours. Occasional rain showers.
Pros: Best for outdoor activities, parks, and longer days.
Cons: Peak tourist season, higher prices, potentially more crowded.
Spring (April-May):
Weather: Mild and pleasant. Average temperatures: 5-15°C (41-59°F). Nature is blooming.
Pros: Fewer crowds than summer, beautiful scenery, good for walking.
Cons: Can still be a bit chilly, unpredictable rain.
Autumn (September-October):
Weather: Crisp and cool. Average temperatures: 5-15°C (41-59°F). Beautiful autumn colors.
Pros: Fewer crowds, lower prices, lovely scenery.
Cons: Days get shorter, can be rainy and windy.
Winter (November-March):
Weather: Cold and short days. Average temperatures: -1 to 5°C (30-41°F). Snow is possible but not guaranteed.
Pros: Christmas markets (late Nov-Dec), cozy atmosphere.
Cons: Very cold, short daylight hours, many outdoor attractions may be closed or have limited hours. Not ideal for very young children.
Recommendation for your family: Aim for late May, June, or September. You'll experience pleasant weather suitable for outdoor activities without the extreme crowds or cold.

3. Suggested Itinerary (7 Days in and around Copenhagen)
This itinerary focuses on Copenhagen and includes day trips, keeping young children's needs in mind (shorter travel times, frequent breaks, engaging activities).

Assumptions:

You'll fly into Copenhagen Airport (CPH).
You'll use public transport (trains, metro, buses) and possibly taxis or ride-sharing services. Copenhagen is very stroller-friendly.
Day 1: Arrival in Copenhagen & Nyhavn Charm

Morning/Afternoon: Arrive at Copenhagen Airport (CPH). Take the train or metro to your hotel. Check in and allow the children to rest.
Late Afternoon: Gentle exploration of the city center. Head to Nyhavn, the iconic harbor with colorful buildings. Enjoy a leisurely walk along the canal.
Evening: Dinner at a family-friendly restaurant near Nyhavn or your hotel. Consider an early night.
Day 2: Tivoli Gardens & City Exploration

Morning: Visit Tivoli Gardens. It's not just an amusement park; it's a beautiful garden with rides suitable for all ages, including many for toddlers. It's magical even during the day.
Lunch: Have lunch inside Tivoli Gardens.
Afternoon: Explore the area around Tivoli, perhaps a walk through Rådhuspladsen (City Hall Square).
Evening: Dinner. Consider a restaurant that offers simple Danish cuisine.
Day 3: Royal Residences & Little Mermaid

Morning: Visit Rosenborg Castle (exterior and gardens are lovely, interior might be less engaging for very young kids, but the crown jewels are impressive). Afterwards, stroll through the King's Garden (Kongens Have), a great place for kids to run around.
Lunch: Picnic in the park or a café nearby.
Afternoon: Head towards the waterfront to see the Little Mermaid statue. It's a bit of a walk, so you might consider a bus or taxi. Enjoy the harbor views.
Evening: Relax at the hotel or find a playground.
Day 4: Day Trip to Louisiana Museum of Modern Art & Coastal Beauty

Morning: Take a scenic train ride (approx. 35-40 minutes) north of Copenhagen to Humlebæk for the Louisiana Museum of Modern Art. While it's an art museum, it's renowned for its stunning architecture, beautiful park setting right by the sea, and excellent children's wing. The grounds are perfect for toddlers to explore.
Lunch: Enjoy lunch at the museum's café with beautiful sea views.
Afternoon: Explore the museum's sculpture garden and the surrounding coastal area.
Evening: Return to Copenhagen for dinner.
Day 5: Experimentarium & Family Fun

Morning/Afternoon: Visit Experimentarium, a fantastic interactive science museum located in Hellerup (easily accessible by train/bus). It's incredibly engaging for children of all ages, with hands-on exhibits that spark curiosity. You can easily spend 3-4 hours here.
Lunch: Have lunch at the Experimentarium café or a nearby spot.
Late Afternoon: Explore the charming harbor area of Hellerup or head back to Copenhagen for some downtime.
Evening: Dinner.
Day 6: Canal Tour & Danish Design

Morning: Take a Canal Tour departing from Nyhavn or Gammel Strand. This is a relaxed way to see many of Copenhagen's landmarks from the water, which kids often enjoy.
Lunch: Lunch in the city center.
Afternoon: Visit the Designmuseum Danmark (exterior and shop might be enough if children are restless) or enjoy a relaxed walk through the city streets, perhaps visiting a toy store or a nice park like Ørstedsparken. Alternatively, revisit a favorite spot or find a local playground.
Evening: Farewell dinner.
Day 7: Departure

Morning: Enjoy a final Danish breakfast. Depending on your flight schedule, you might have time for some last-minute souvenir shopping or a quick visit to a local park.
Afternoon: Head to Copenhagen Airport (CPH) for your flight back to Pune.
4. Hotel Suggestions (Family-Friendly in Copenhagen)
When traveling with young children, look for hotels that offer:

Spacious rooms or family suites.
Cribs/cots availability.
Kid-friendly amenities (play areas, pools if available).
Good location with easy access to public transport and attractions.
Proximity to parks or playgrounds.
Here are a few options in different budget ranges:

Luxury / Higher Budget:

Hotel d'Angleterre: Iconic luxury hotel in the heart of the city. Offers spacious rooms and top-notch service. While not explicitly a "kid's hotel," their service can accommodate families well.
Nimb Hotel: Located within Tivoli Gardens, offering a unique experience with beautifully designed rooms and excellent dining. The proximity to Tivoli is a huge advantage.
Mid-Range / Good Value:

Scandic Palace Hotel: Central location on City Hall Square. Scandic hotels are generally known for being family-friendly, often providing free breakfasts for kids and offering amenities like children's menus and play corners.
Wakeup Copenhagen (Various Locations): Modern, stylish, and budget-friendly. They offer compact but well-designed rooms. Some locations might have family rooms or connecting rooms. Good for those who prioritize location and modern amenities over extensive services.
Adina Apartment Hotel Copenhagen: Offers spacious apartments with kitchenettes, which can be incredibly convenient when traveling with young children for preparing snacks or simple meals. Good location near the waterfront.
Budget-Friendly (while still comfortable):

Cabinn (Various Locations): Offers very compact rooms at affordable prices. Might be a good option if you plan to spend most of your time out and about. Check for family rooms, though they will still be basic.
A&O Copenhagen Nørrebro: Hostel/budget hotel with private rooms and family rooms. Can be a good option for value, often with common areas and kitchens.
Key Considerations for Booking:

Family Rooms: Always look for "family rooms" or book two rooms if necessary.
Location: Aim for areas like Indre By (City Center), Vesterbro (near Tivoli), or Østerbro (quieter, good parks).
Breakfast: Having breakfast included can simplify mornings with kids.
Reviews: Read recent reviews from other families.
5. Budget Estimation (10,00,000 INR for 7 Days)
This is a generous budget, allowing for comfortable travel, good accommodation, and plenty of activities. Exchange rate used: 1 INR ≈ 0.085 DKK (Danish Krone) or 1 EUR ≈ 7.45 DKK. So, 10,00,000 INR ≈ 85,000 DKK ≈ €11,400.

Here's a breakdown (estimates, can be adjusted):

Flights (Pune to Copenhagen, Round Trip for 4): ₹2,80,000 - ₹4,00,000 (Highly variable based on booking time, airline, and season. Book in advance!)
Accommodation (7 nights, Mid-range to Upper Mid-range Family-friendly hotel): ₹1,75,000 - ₹2,80,000 (₹25,000 - ₹40,000 per night)
Visa Fees & Travel Insurance: ₹40,000 - ₹60,000
Local Transport (Public transport passes, occasional taxis): ₹50,000 - ₹70,000
Activities & Entrance Fees (Tivoli, Experimentarium, Canal Tour, etc.): ₹1,00,000 - ₹1,50,000 (Can be managed for less if you prioritize free parks and walking.)
Food & Dining (Mix of restaurants, cafes, and some self-catering if possible): ₹2,00,000 - ₹2,50,000 (≈ ₹30,000 - ₹35,000 per day)
Miscellaneous (Souvenirs, personal expenses, buffer): ₹50,000 - ₹1,00,000
Total Estimated: ₹9,45,000 - ₹13,10,000

Tips for Managing Budget:

Book Flights and Hotels Early: This is crucial for getting better prices, especially for family rooms.
Consider Copenhagen Card: If you plan to visit many paid attractions and use public transport extensively, the Copenhagen Card might offer good value. Calculate if it makes sense for your planned activities.
Picnics: Utilize Denmark's beautiful parks for picnics. Supermarkets like Netto, Rema 1000, and Føtex offer affordable options.
Free Activities: Copenhagen has many free parks, playgrounds, and charming streets to explore.
Travel Season: Traveling slightly off-peak (late May/early Sept) can offer better prices than July/August.
Important Notes for Traveling with Young Children:
Pacing: Don't overschedule. Allow for naps, downtime, and spontaneous playground visits.
Stroller/Carrier: Essential for getting around. Copenhagen is very stroller-friendly.
Snacks & Drinks: Carry familiar snacks and water.
Playgrounds: Denmark has fantastic public playgrounds. Incorporate these into your daily activities.
Public Transport: Efficient and child-friendly. Most stations have elevators, and buses have designated spaces for strollers.
Weather Gear: Pack layers, waterproof jackets, and comfortable walking shoes. Even in summer, evenings can be cool.
This plan provides a solid framework. You can adjust it based on your children's specific interests and energy levels. Have a wonderful family vacation in Denmark!